In [ ]:
%matplotlib inline
import openpathsampling as paths
import numpy as np

As always, we load things from files so we don't have to set them up again.

In [ ]:
old = paths.Storage("mistis.nc", 'r')
engine = old.engines[0]
network = old.networks[0]
states = set(network.initial_states + network.final_states)

In [ ]:
# must ensure that the diskcache is disabled in order to save,
# otherwise it looks for things that aren't there!
cvs = old.cvs[:]
for cv in cvs:
    cv.disable_diskcache()

The `flux_pairs` variable is a list of 2-tuples, where the first element is the state we're calculating the flux out of, and the second element is the interface we're calculating the flux through.

In [ ]:
flux_pairs = [(t.stateA, t.interfaces[0]) for t in network.transitions.values()]

Set up the simulation and run it!

In [ ]:
sim = paths.DirectSimulation(
    storage=None,
    engine=engine,
    states=states,
    flux_pairs=flux_pairs,
    initial_snapshot=old.snapshots[0]
)

In [ ]:
%%time
sim.run(150000) # 30 sec
#sim.run(1500000) # 6 min
#sim.run(15000000) # 60 min
#sim.run(150000000) # 10 hr
#sim.run(800000000) # 2 days

Now we move on to the analysis.

In [ ]:
sim.rate_matrix

In [ ]:
sim.n_transitions

In [ ]:
fluxes = sim.fluxes
for f in fluxes:
    print f[0].name, f[1].name, fluxes[f]

In [ ]:
sim.n_flux_events

In [ ]:
sim.results

In [ ]:
output = paths.Storage("direct_simulation.nc", 'w')
output.save(old.snapshots[0])
output.save(sim)
output.tag['direct_results'] = sim.results
output.close()